# Preparation



## Only for Colab

### Unzip the dataset

In [1]:
%%script false
# unzip the zip dataset
import zipfile
!unzip /content/sample_data/a.zip

Couldn't find program: 'false'


In [2]:
%%script false
# Install TensorFlow v2 only in Colab
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Couldn't find program: 'false'


## Initial import

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

# Upload and preprocessing



In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                       validation_split=0.2)

# Create validation and test ImageDataGenerator objects
test_data_gen = ImageDataGenerator(rescale=1./255)


### Create generators to read images from dataset directory

**.flow_from_directory:**
returns a DirectoryIterator yielding tuples of (x, y) where x is a numpy array containing a batch of images 
with shape (batch_size, *target_size, channels) and y is a numpy array of corresponding labels.

In [5]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, 'Classification_Dataset')
print (dataset_dir)
# img

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes=20
class_list = ['owl', 'galaxy','lightning', 'wine-bottle', 't-shirt', 'waterfall', 'sword', 'school-bus', 'calculator', 'sheet-music', 'airplanes', 'lightbulb', 'skyscraper', 'mountain-bike', 'fireworks', 'computer-monitor', 'bear', 'grand-piano', 'kangaroo', 'laptop']

# Training 
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               classes=class_list,
                                               batch_size=bs,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                              subset='training')  # targets are directly converted into one-hot vectors

# Test
test_dir = os.path.join(dataset_dir,'test')
test_gen = test_data_gen.flow_from_directory(
        test_dir,
        target_size=(256, 256),
        class_mode=None,
        shuffle=False,
        batch_size=1)


# Validation
valid_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=class_list,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED,
                                               subset='validation')

C:\Users\Roncax\Desktop\Git\tensorflow_exercises\notebooks\Classification_Dataset
Found 1247 images belonging to 20 classes.
Found 500 images belonging to 1 classes.
Found 307 images belonging to 20 classes.


## Create Dataset objects

**tf.data.Dataset.from_generator:**
creates a Dataset whose elements are generated by generator.
The generator argument must be a callable object that returns an object that supports the iter() protocol (e.g. a generator function). The elements generated by generator must be compatible with the given output_types and (optional) output_shapes arguments.

In [6]:
# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Shuffle (Already done in generator..)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()


In [7]:
%%script false
# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

Couldn't find program: 'false'


### Data augmentation

In [8]:
%%script false
# Let's test data augmentation
# ----------------------------
import time
import matplotlib.pyplot as plt

%matplotlib notebook

fig = plt.figure()
ax = fig.gca()
fig.show()

iterator = iter(train_dataset)

for _ in range(1000):
    augmented_img, target = next(iterator)
    augmented_img = augmented_img[0]   # First element
    augmented_img = augmented_img * 255  # denormalize
    
    plt.imshow(np.uint8(augmented_img))
    fig.canvas.draw()
    time.sleep(1)

Couldn't find program: 'false'


# Processing


In [9]:
%%script false
# Check that is everything is ok..

iterator = iter(train_dataset)
sample, target = next(iterator)

# Just for visualization purpouses
sample = sample[18, ...]  # select first image in the batch
sample = sample * 255  # denormalize

from PIL import Image
img = Image.fromarray(np.uint8(sample))
img = img.resize([128, 128])
img

# target[0]  # select corresponding target

Couldn't find program: 'false'


In [10]:
# Keras Model subclassing 
# -----------------------

# Please note that there are many ways to implement a CNN
# Here subclassing is used just for teaching purposes, but you
# can simply write every single layer as usual using Model or Sequential..

# Create convolutional block
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()  # we can specify the activation function directly in Conv2D
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [11]:
# Create Model
# ------------

depth = 5
start_f = 8
num_classes = 20

class CNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(units=512, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x
    
# Create Model instance
model = CNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

In [12]:
# Visualize created model as a table
model.feature_extractor.summary()

# Visualize initialized weights
# model.weights[0]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block (ConvBlock)       multiple                  224       
_________________________________________________________________
conv_block_1 (ConvBlock)     multiple                  1168      
_________________________________________________________________
conv_block_2 (ConvBlock)     multiple                  4640      
_________________________________________________________________
conv_block_3 (ConvBlock)     multiple                  18496     
_________________________________________________________________
conv_block_4 (ConvBlock)     multiple                  73856     
Total params: 98,384
Trainable params: 98,384
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training with callbacks

In [14]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'challenge_1')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)
# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Train for 156 steps, validate for 39 steps
Epoch 1/100
156/156 [==============================] - 23s 147ms/step - loss: 2.8939 - accuracy: 0.0962 - val_loss: 2.6689 - val_accuracy: 0.1401
Epoch 2/100
156/156 [==============================] - 22s 140ms/step - loss: 2.4862 - accuracy: 0.2470 - val_loss: 2.2399 - val_accuracy: 0.3094
Epoch 3/100
156/156 [==============================] - 22s 138ms/step - loss: 2.0408 - accuracy: 0.3697 - val_loss: 2.0727 - val_accuracy: 0.3583
Epoch 4/100
156/156 [==============================] - 21s 136ms/step - loss: 1.6698 - accuracy: 0.4900 - val_loss: 2.1557 - val_accuracy: 0.3844
Epoch 5/100
156/156 [==============================] - 21s 134ms/step - loss: 1.2503 - accuracy: 0.6263 - val_loss: 2.1547 - val_accuracy: 0.4007
Epoch 6/100
156/156 [==============================] - 22s 139ms/step - loss: 0.7399 - accuracy: 0.7634 - val_loss: 2.6434 - val_accuracy: 0.4235
Epoch 7/100
156/156 [==============================] - 21s 136ms/step - loss: 0.4

In [15]:
prediction = model.predict_generator(test_gen,
                                    verbose=1)
prediction.shape
print(prediction[1])

500/500 [==============================] - 13s 25ms/step


(500, 20)

[3.1503391e-09 1.0179600e-17 3.0913227e-16 3.3462948e-11 3.5354009e-16
 2.3382073e-04 7.2577585e-08 2.0278328e-17 1.3346029e-11 1.4233487e-19
 1.0882183e-26 2.4542083e-12 3.1343549e-16 2.3303782e-13 1.2002391e-14
 2.9741807e-14 5.7226978e-04 2.7450008e-26 9.9919385e-01 1.7180558e-15]


In [16]:
image=0
relations = {} 
image_names = []
images_class_num= []
from PIL import Image
for img_pred in prediction:
    class_num=img_pred.argmax()
    print(image,max(img_pred), class_num, class_list[class_num])
    # img = Image.open(test_dir + '\\' + test_gen.filenames[image])
    # img
    image_name = test_gen.filenames[image].replace('sub\\','')
    relations[image_name] = class_num
    image_names.append(image_name)
    images_class_num.append(class_num)
    
    image+=1
    
print (len(relations))

0 0.9999161 5 waterfall
1 0.99919385 18 kangaroo
2 0.9283391 14 fireworks
3 1.0 15 computer-monitor
4 0.9999999 14 fireworks
5 1.0 14 fireworks
6 0.9044057 13 mountain-bike
7 1.0 7 school-bus
8 0.9999013 10 airplanes
9 0.9993305 15 computer-monitor
10 0.59523773 2 lightning
11 0.49908912 0 owl
12 0.65128773 18 kangaroo
13 0.9368217 8 calculator
14 0.9991617 13 mountain-bike
15 0.9987196 5 waterfall
16 0.999992 9 sheet-music
17 0.9999943 5 waterfall
18 0.91531163 16 bear
19 1.0 7 school-bus
20 0.9998416 7 school-bus
21 0.9997794 10 airplanes
22 0.45709234 5 waterfall
23 1.0 12 skyscraper
24 1.0 7 school-bus
25 0.87342256 13 mountain-bike
26 0.99999964 14 fireworks
27 0.69645387 16 bear
28 1.0 10 airplanes
29 0.8899331 9 sheet-music
30 0.9998901 7 school-bus
31 1.0 15 computer-monitor
32 0.9999789 12 skyscraper
33 1.0 14 fireworks
34 0.9999938 4 t-shirt
35 1.0 3 wine-bottle
36 1.0 16 bear
37 0.9542202 8 calculator
38 0.77431047 5 waterfall
39 0.9999981 17 grand-piano
40 0.9129893 19 lapt

In [22]:
import csv
i=0
w = csv.writer(open("output.csv", "w"))
w.writerow(["Id", "Category"])
for key, val in relations.items():
    i=i+1
    print(i)
    w.writerow([key, val])
    

13

1


13

2


17

...

16

499


15

500


15